In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
import pandas as pd
import os, json

def save_json(file, data):
    file_path = f"{file}.json"
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            existing_data = json.load(f)
        existing_data.extend(data)
        with open(file_path, "w") as f:
            f.write(json.dumps(existing_data, indent=2))
    else:
        try:
            with open(file_path, "w") as f:
                f.write(json.dumps(data, indent=2))
        except Exception as e:
            print(e)

In [17]:
import requests
from bs4 import BeautifulSoup
import json

# Replace with your target URL
url = "https://biblicalcounseling.com/resource-library/topic-index/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
categories = {}
category_sections = soup.find_all('li', class_='collection__item')
for section in category_sections:
    category_name = section.find('a',class_="collection__item--title").text.strip()        
    links = section.find_all('li', class_='term_child')
    # Store links for this category
    categories[category_name] = [{
        'title': link.text.strip(),
        'url': link.find("a")["href"]
    } for link in links]


In [25]:
def scrape_content():
    url = "https://biblicalcounseling.com/resource-library/podcast-episodes/psychedelic-microdosing/"
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36'}

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    header_component = soup.find("div", class_= "card-content")
    title = header_component.find("h1").text.strip() if header_component.find("h1") else " " 
    subtitle = header_component.find("h3").text.strip() if header_component.find("h3") else " " 
    question = header_component.find("p").text.strip() if header_component.find("p") else " " 
    authors = header_component.find("div", class_="authors-wrapper").text.strip() if header_component.find("div", class_="authors-wrapper") else " "
    content = soup.find("div", class_="article-content").text.strip()
    source = url

    return {
            "title": title,
            "subtitle": subtitle,
            "author": authors,
            "question": question,
            "body": content,
            "source": source
        }


{'title': 'Psychedelic Microdosing', 'subtitle': 'Truth in Love 488', 'author': 'Dale Johnson  Mark Shaw', 'question': 'How do we respond biblically to the medical use of Psychedelics?', 'body': 'Dale Johnson: Today on the podcast, I have with me, Dr. Mark Shaw. He’s a pastor of counseling at the Nocatee\xa0campus of First Baptist Church in Jacksonville, Florida. He’s the founder of the Addiction Connection, a non-profit equipping ministry for addiction biblical counselors. He’s the author of 28 Publications, including The Heart of Addiction, and the latest one is called Destigmatizing Addiction, Hope through the Gospel. His passion is to see the local church offer compassionate biblical counseling to hurting souls. Mark, Welcome to podcast. You’re no stranger here to our listeners. Thank you for being with us today.\nMark Shaw: \xa0Thank you, Dale. Great to be with you.\nDale Johnson: Now, it is an interesting topic that you’ve chosen here today, I’m looking forward to diving in here.

In [18]:
categories = list(categories.items())[1:]



{'ACBC': [{'title': 'Announcements and Events 33',
   'url': '/resource-library/topic-index/announcements-and-events'},
  {'title': 'Book Reviews 59',
   'url': '/resource-library/topic-index/book-reviews'},
  {'title': 'Certification 6',
   'url': '/resource-library/topic-index/certification'}],
 'Biblical Counseling and Psychology': [{'title': 'Medical Issues 9',
   'url': '/resource-library/topic-index/medical-issues'},
  {'title': 'Mental Health 19',
   'url': '/resource-library/topic-index/mental-health'},
  {'title': 'Psychological Theories 24',
   'url': '/resource-library/topic-index/psychological-theories'}],
 'Counseling Practice': [{'title': 'Counseling Methodology 53',
   'url': '/resource-library/topic-index/counseling-methodology'},
  {'title': 'How to Begin a Counseling Center 16',
   'url': '/resource-library/topic-index/how-to-begin-a-counseling-center'},
  {'title': 'Legal Issues in Counseling 12',
   'url': '/resource-library/topic-index/legal-issues-in-counseling'}]